# Example: Bar with elastic mass
Started in class, 6-Apr-2021.  

The uniform bar in the vertical plane has a point-mass attached via a spring at the tip.  The two coordinates of the problem are $q = (x,\theta)$.  There is an external force $f$ that acts vertically.  The torsion spring has an unstretched angle $\theta_0$, and the linear spring has an unstretched length of $x_0$.

![Bad sketch of the system](system_sketch.svg)


## Problem setup and constants

In [1]:
using SymPy

In [2]:
M, m, g, k, kt, c, a, lG, lT, lf ,t, θ0, x0 = symbols("M, m, g, k, k_t, c, a, l_G, l_T, l_f , t, theta_0, x_0", real=true, positive=true)
x = SymFunction("x")(t)
θ = SymFunction("theta")(t)
f = SymFunction("f")(t)
ihat = [1,0,0]
jhat = [0,1,0]
khat = [0,0,1]
;

Now let's build position vectors to each important point in the system.  Each position is with respect to the fixed-point at $O$.

In [3]:
rG = lG*(cos(θ)*ihat + sin(θ)*jhat)

3-element Vector{Sym}:
 l_G*cos(theta(t))
 l_G*sin(theta(t))
                 0

In [4]:
rf = lf*(cos(θ)*ihat + sin(θ)*jhat)

3-element Vector{Sym}:
 l_f*cos(theta(t))
 l_f*sin(theta(t))
                 0

In [5]:
ra = a*(cos(θ)*ihat + sin(θ)*jhat)

3-element Vector{Sym}:
 a*cos(theta(t))
 a*sin(theta(t))
               0

In [6]:
rB = lT*(cos(θ)*ihat + sin(θ)*jhat)

3-element Vector{Sym}:
 l_T*cos(theta(t))
 l_T*sin(theta(t))
                 0

In [7]:
rm = rB - x*jhat

3-element Vector{Sym}:
        l_T*cos(theta(t))
 l_T*sin(theta(t)) - x(t)
                        0

## Let's build the Lagrangian

$$ T = \frac{1}{2} M \dot{\vec{r}}_G \cdot \dot{\vec{r}}_G + \frac{1}{2} J_G \dot{\theta}^2 + \frac{1}{2} m \dot{\vec{r}}_m\cdot \dot{\vec{r}}_m$$

$$ V = \frac{1}{2} k_t \left( \theta - \theta_0 \right)^2 + \frac{1}{2} k \left( x - x_0 \right)^2  + M g \vec{r}_G \cdot \hat{j} + m g \vec{r}_m \cdot \hat{j}$$

In [8]:
dot(a,b) = sum(a .* b)
JG = 1//12*M*lT^2
T = 1//2*M*dot( diff.(rG,t), diff.(rG,t) ) + 1//2*JG*diff(θ,t)^2 + 1//2*m*dot( diff.(rm,t), diff.(rm,t)) |> simplify

                     2                        2     /                   2     
     2 /d           \         2 /d           \      |   2 /d           \      
M*l_G *|--(theta(t))|    M*l_T *|--(theta(t))|    m*|l_T *|--(theta(t))|  - 2*
       \dt          /           \dt          /      \     \dt          /      
---------------------- + ---------------------- + ----------------------------
          2                        24                                         

                                                    2\
                  d            d          /d       \ |
l_T*cos(theta(t))*--(theta(t))*--(x(t)) + |--(x(t))| |
                  dt           dt         \dt      / /
------------------------------------------------------
            2                                         

In [9]:
V = 1//2*kt*(θ - θ0)^2 + 1//2*k*(x - x0)^2 + M*g*dot(rG, jhat) + m*g*dot(rm, jhat)

                                                                        2     
                                                         k*(-x_0 + x(t))    k_
M*g*l_G*sin(theta(t)) + g*m*(l_T*sin(theta(t)) - x(t)) + ---------------- + --
                                                                2             

                       2
t*(-theta_0 + theta(t)) 
------------------------
          2             

In [10]:
L = T - V

                                              2                        2      
                              2 /d           \         2 /d           \       
                         M*l_G *|--(theta(t))|    M*l_T *|--(theta(t))|       
                                \dt          /           \dt          /       
-M*g*l_G*sin(theta(t)) + ---------------------- + ---------------------- - g*m
                                   2                        24                

                                                                              
                                                                              
                                             2                            2   
                              k*(-x_0 + x(t))    k_t*(-theta_0 + theta(t))    
*(l_T*sin(theta(t)) - x(t)) - ---------------- - -------------------------- + 
                                     2                       2                

  /                   2                           

## 

In [11]:
rc = a*(cos(θ)*ihat + sin(θ)*jhat)
D = 1//2*c*dot(diff.(rc,t), jhat)^2

                                  2
 2      2           /d           \ 
a *c*cos (theta(t))*|--(theta(t))| 
                    \dt          / 
-----------------------------------
                 2                 

## Lagrange's Equation

In [12]:
lag = [ diff(L, diff(q,t),t) - diff(L,q) + diff(D, diff(q,t)) |> simplify for q in [x,θ] ]

2-element Vector{Sym}:
                                                                                                                                               -g*m - k*(x_0 - x(t)) + m*(l_T*sin(theta(t))*Derivative(theta(t), t)^2 - l_T*cos(theta(t))*Derivative(theta(t), (t, 2)) + Derivative(x(t), (t, 2)))
 M*g*l_G*cos(theta(t)) + M*l_G^2*Derivative(theta(t), (t, 2)) + M*l_T^2*Derivative(theta(t), (t, 2))/12 + a^2*c*cos(theta(t))^2*Derivative(theta(t), t) + g*l_T*m*cos(theta(t)) - k_t*theta_0 + k_t*theta(t) + l_T^2*m*Derivative(theta(t), (t, 2)) - l_T*m*cos(theta(t))*Derivative(x(t), (t, 2))

Now let's work on the generalized forces $Q_k = \vec{F} \cdot \frac{\partial \vec{r}_f}{\partial q_k}$

In [13]:
F = f*jhat
dot(F, diff.(rf, x) )

0

In [14]:
dot(F, diff.(rf, θ) )

l_f*f(t)*cos(theta(t))

In [15]:
Q = [ dot(F, diff.(rf, q) ) for q in [x,θ] ]

2-element Vector{Sym}:
                      0
 l_f*f(t)*cos(theta(t))

## Hamilton's Equation

First, let's compute the transforms between $\dot q$ and $p$

In [16]:
p1,p2 = symbols("p_1, p_2", real=true)

(p_1, p_2)

In [28]:
eqn1 = [ Eq(p1, diff(L, diff(x,t)) ),
    Eq(p2, diff(L, diff(θ,t)) ) ]

2-element Vector{Sym}:
                                                                             Eq(p_1, m*(-2*l_T*cos(theta(t))*Derivative(theta(t), t) + 2*Derivative(x(t), t))/2)
 Eq(p_2, M*l_G^2*Derivative(theta(t), t) + M*l_T^2*Derivative(theta(t), t)/12 + m*(2*l_T^2*Derivative(theta(t), t) - 2*l_T*cos(theta(t))*Derivative(x(t), t))/2)

In [18]:
solp = solve(eqn1, [diff(x,t), diff(θ,t)])

Dict{Any, Any} with 2 entries:
  Derivative(x(t), t)     => (-12*M*l_G^2*p_1 - M*l_T^2*p_1 - 12*l_T^2*m*p_1 - …
  Derivative(theta(t), t) => (-12*l_T*p_1*cos(theta(t)) - 12*p_2)/(-12*M*l_G^2 …

Now, let's build the Hamiltonian

In [19]:
H = T + V |> subs(solp) |> simplify

                                                                              
       /      2      2\                              2          2  2          
12*M*m*\12*l_G  + l_T /*(l_T*p_1*cos(theta(t)) + p_2)  + 144*l_T *m *(l_T*p_1*
------------------------------------------------------------------------------
                                                                              
                                                                              
                                                                              

                                                                              
                    2                                          /        2     
cos(theta(t)) + p_2)  - 24*l_T*m*(l_T*p_1*cos(theta(t)) + p_2)*\12*M*l_G *p_1 
------------------------------------------------------------------------------
                                                                              
                                                   

Let's convert the damping in terms of $p$

In [20]:
Qdamping = - [ diff(D, diff(q,t)) |> subs(solp) for q in [x,θ] ]

2-element Vector{Sym}:
                                                                                                                             0
 -a^2*c*(-12*l_T*p_1*cos(theta(t)) - 12*p_2)*cos(theta(t))^2/(-12*M*l_G^2 - M*l_T^2 + 12*l_T^2*m*cos(theta(t))^2 - 12*l_T^2*m)

Now let's build Hamilton's equation
$$ \dot{p}_k = -\frac{\partial H}{\partial q_k} + Q_k + Q_{k,\text{damping}}$$
$$ \dot{q}_k = \frac{\partial H}{\partial p_k} $$


In [21]:
pdot = [ -diff(H, q) |> simplify for q in [x,θ] ] + Q  + Qdamping

2-element Vector{Sym}:
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 

In [22]:
qdot = [ diff(H, p) for p in [p1,p2]]

2-element Vector{Sym}:
 (24*M*l_T*m*(12*l_G^2 + l_T^2)*(l_T*p_1*cos(theta(t)) + p_2)*cos(theta(t)) + 288*l_T^3*m^2*(l_T*p_1*cos(theta(t)) + p_2)*cos(theta(t)) - 24*l_T^2*m*(12*M*l_G^2*p_1 + M*l_T^2*p_1 + 12*l_T^2*m*p_1 + 12*l_T*m*p_2*cos(theta(t)))*cos(theta(t))^2 - 24*l_T*m*(l_T*p_1*cos(theta(t)) + p_2)*(12*M*l_G^2 + M*l_T^2 + 12*l_T^2*m)*cos(theta(t)) + (24*M*l_G^2 + 2*M*l_T^2 + 24*l_T^2*m)*(12*M*l_G^2*p_1 + M*l_T^2*p_1 + 12*l_T^2*m*p_1 + 12*l_T*m*p_2*cos(theta(t))))/(2*m*(12*M*l_G^2 + M*l_T^2 + 12*l_T^2*m*sin(theta(t))^2)^2)
                                                                                                                                                                                                                                                                                     (12*M*m*(12*l_G^2 + l_T^2)*(2*l_T*p_1*cos(theta(t)) + 2*p_2) - 288*l_T^2*m^2*(l_T*p_1*cos(theta(t)) + p_2)*cos(theta(t))^2 + 144*l_T^2*m^2*(2*l_T*p_1*cos(theta(t)) + 2*p_2))/(2*m*(12*M*l_G^

Exporting can be done systematically using the `sympy.octave_code` to get to Matlab, or `sympy.julia_code` to export Julia code.

In [23]:
sympy.octave_code(pdot) |> println

% Not supported in Octave:
% f
% theta
% x
[g.*m + k.*(x_0 - x(t)); -a.^2.*c.*(-12*l_T.*p_1.*cos(theta(t)) - 12*p_2).*cos(theta(t)).^2./(-12*M.*l_G.^2 - M.*l_T.^2 + 12*l_T.^2.*m.*cos(theta(t)).^2 - 12*l_T.^2.*m) + l_f.*f(t).*cos(theta(t)) + (24*l_T.^2.*(12*M.*m.*(12*l_G.^2 + l_T.^2).*(l_T.*p_1.*cos(theta(t)) + p_2).^2 + 144*l_T.^2.*m.^2.*(l_T.*p_1.*cos(theta(t)) + p_2).^2 - 24*l_T.*m.*(l_T.*p_1.*cos(theta(t)) + p_2).*(12*M.*l_G.^2.*p_1 + M.*l_T.^2.*p_1 + 12*l_T.^2.*m.*p_1 + 12*l_T.*m.*p_2.*cos(theta(t))).*cos(theta(t)) + m.*(12*M.*l_G.^2 + M.*l_T.^2 + 12*l_T.^2.*m.*sin(theta(t)).^2).^2.*(2*M.*g.*l_G.*sin(theta(t)) + 2*g.*m.*(l_T.*sin(theta(t)) - x(t)) + k.*(x_0 - x(t)).^2 + k_t.*(theta_0 - theta(t)).^2) + (12*M.*l_G.^2.*p_1 + M.*l_T.^2.*p_1 + 12*l_T.^2.*m.*p_1 + 12*l_T.*m.*p_2.*cos(theta(t))).^2).*sin(theta(t)).*cos(theta(t)) + (12*M.*l_G.^2 + M.*l_T.^2 + 12*l_T.^2.*m.*sin(theta(t)).^2).*(12*M.*l_T.*p_1.*(12*l_G.^2 + l_T.^2).*(l_T.*p_1.*cos(theta(t)) + p_2).*sin(theta(t)) + 144*l_T.^3.

## Linearizing the system of equations

This is a *just because we can* type of exercise.  I'm going to prescribe the equilibrium to $\theta_{eq} = 0$, and determine the other relationships.  The state-space form of 
$$ \dot{z} = A z + B u $$
where the states are the linearized parts of $(p_1, p_2, x, \theta)$

In [43]:
xeq = symbols("x_eq", real=true)
eqn_eq = [pd |> subs(p1=>0, p2=>0, f=>0, θ=>0, x=>xeq) for pd in pdot]

2-element Vector{Sym}:
             g*m + k*(x_0 - x_eq)
 -M*g*l_G - g*l_T*m + k_t*theta_0

The $\dot{q}_k$ equations do not give any info

In [39]:
[ qd |> subs(p1=>0, p2=>0, f=>0, θ=>0) for qd in qdot ]

2-element Vector{Sym}:
 0
 0

If we want to know the unstretched length of the springs to ensure the equilbrium point is where we want it to be, then we can solve the above equations for $(x_0, \theta_0)$

In [45]:
solve( eqn_eq, [x0, θ0] )

Dict{Any, Any} with 2 entries:
  theta_0 => (M*g*l_G + g*l_T*m)/k_t
  x_0     => (-g*m + k*x_eq)/k

Now let's compute the Jabocian (standard Taylor series)

In [49]:
A = [ diff(ff,z) |> subs(p1=>0, p2=>0, f=>0, θ=>0, x=>xeq) |> simplify for ff in vcat(pdot, qdot), z in [p1,p2,x,θ] ]

4×4 Matrix{Sym}:
                                                            0  …  -k     0
                         -12*a^2*c*l_T/(M*(12*l_G^2 + l_T^2))      0  -k_t
 (12*M*l_G^2 + M*l_T^2 + 12*l_T^2*m)/(M*m*(12*l_G^2 + l_T^2))      0     0
                                12*l_T/(M*(12*l_G^2 + l_T^2))      0     0

In [52]:
B = [ diff(ff, f) |> subs(p1=>0, p2=>0, f=>0, θ=>0, x=>xeq) for ff in vcat(pdot, qdot) ]

4-element Vector{Sym}:
   0
 l_f
   0
   0